# Finding and Ordering Imagery for the Largest Active Fire on Earth

In this notebook, we'll combine NASA's FIRMS (Fire Information for Resource Management System) active fire data with Google Earth Engine and Planet imagery to:

1. **Load today's active fires** from the MODIS or VIIRS instruments via FIRMS
2. **Identify the largest fire** by grouping nearby fire pixels into connected clusters
3. **Polygonize the fire boundary** to create an area of interest (AOI)
4. **Search Planet's catalog** for recent PlanetScope imagery over that AOI
5. **Order and visualize** the imagery directly in this notebook using geemap

This workflow demonstrates real-world remote sensing analysis: detecting an event from coarse hotspot data, defining a precise AOI, and acquiring high-resolution commercial imagery for detailed assessment.

## APIs and Tools

- **[Google Earth Engine Python API](https://developers.google.com/earth-engine/guides/python_install)** – for geospatial analysis and data processing
- **[geemap](https://geemap.org/)** – interactive mapping library for Earth Engine in Python
- **[Planet SDK for Python](https://planet-sdk-for-python-v2.readthedocs.io/)** – for programmatic access to Planet's data catalog
- **[Planet Orders API](https://developers.planet.com/docs/apis/orders/)** – for ordering and delivering Planet imagery

## Datasets

- **[FIRMS Active Fire Data](https://firms.modaps.eosdis.nasa.gov/)** – near real-time fire detections from MODIS and VIIRS instruments
- **[PlanetScope Imagery](https://developers.planet.com/docs/data/planetscope/)** – high-resolution (3–5 m) daily imagery from Planet's satellite constellation

## Prerequisites

- Authenticated access to Google Earth Engine (`ee.Authenticate()` and `ee.Initialize()`)
- A Planet API key (sign up at [planet.com](https://www.planet.com/) for a free trial or educational account)
- Python packages: `earthengine-api`, `geemap`, `planet`, `requests`, `geopandas`, `shapely`

Let's get started!

## Installing Required Libraries

Before we begin, we need to install the necessary Python packages for this workflow:

- **earthengine-api** – Google Earth Engine Python API for geospatial analysis
- **geemap** – interactive mapping library that simplifies Earth Engine visualization
- **planet** – Planet SDK for searching and ordering commercial satellite imagery
- **requests** – HTTP library for API calls (e.g., downloading FIRMS data)
- **geopandas** – geospatial extension of pandas for working with vector data
- **shapely** – geometric operations library for creating and manipulating spatial objects

Run the cell below to install or upgrade these packages. The `-q` flag suppresses verbose output, and `--upgrade` ensures you have the latest versions.

**Note:** Some operations in this workflow (particularly Planet API interactions) may benefit from asynchronous execution to handle multiple requests efficiently.

In [13]:
!pip install -q --upgrade earthengine-api leafmap geemap planet requests geopandas shapely asyncio geojson

## 1. Authentication with Google Earth Engine

Before you can use GEE, you need to authenticate your account. There are several ways to do this:

*   **Browser Authentication (Recommended):** This is the easiest method.  GEE will open a browser window to allow you to log in with your Google account.
*   **Service Account:**  More suitable for automated scripts and server-side applications. Requires creating a service account in the Google Cloud Console and downloading credentials.

### 1.1 Browser Authentication (using `ee.Initialize()`)

The following code snippet initializes the Earth Engine API and authenticates your account using browser authentication.  Run this cell *first* to authenticate.

In [14]:
import ee
import geemap as geemap

# Initialize Earth Engine (authenticate if needed)
try:
    ee.Initialize()
except Exception:
    ee.Authenticate()
    ee.Initialize()
# Print a confirmation message to let us know the initialization was successful
print('Earth Engine initialized.')

Earth Engine initialized.


In [ ]:

# Load the FIRMS image collection.
dataset = ee.ImageCollection("FIRMS")

geometry = ee.Geometry.Rectangle([-180, -90, 180, 90])

# Get the most recent image (for visualization)
lastimg = dataset.sort('system:time_start', False).first()

# Select the T21 band from the latest image
fires = lastimg.select('T21').gt(100)

# Compute the number of connected pixels (including itself) in each cluster.
connectedCount = fires.connectedPixelCount(1000, True)

# Visualization styles
firesVis = {
    "min": 100.0,
    "max": 500.0,
    "palette": ["yellow", "orange", "red"]
}

countVis = {
    "min": 0.0,
    "max": 100.0,
    "palette": ["yellow", "orange", "red"]
}

# Calculate the min and max values for each band in the region.
minMax = connectedCount.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=geometry,
    scale=1000,  # Set an appropriate scale for your analysis
    maxPixels=1e9  # Increase maxPixels if needed for large regions
)

# Print the result to the console.
print("Min and Max values:", minMax.getInfo())



Min and Max values: {'T21_max': 147, 'T21_min': 1}


In [16]:

# Visualization styles
firesVis = {
    "min": 100.0,
    "max": 500.0,
    "palette": ["yellow", "orange", "red"]
}

countVis = {
    "min": 0.0,
    "max": 100.0,
    "palette": ["yellow", "orange", "red"]
}

# Visualization: purple -> yellow ramp for connected pixel counts
conn_vis = {
    'min': 0,
    'max': 100,
    'palette': ['#4B0082', '#6A5ACD', '#8A2BE2', '#DA70D6', '#FFD700']
}



m = geemap.Map(center=[0, 0], zoom=2)
m.addLayer(lastimg.select('T21'), firesVis, 'FIRMS T21')

# Add the connectedCount layer to the existing geemap Map
m.addLayer(connectedCount, conn_vis, 'Connected pixels (count) — purple→yellow')

# Display the map (ensure this is the last line in the cell)
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

In [17]:
T21_max = ee.Number(minMax.get('T21_max'))
largest_cluster = connectedCount.eq(T21_max)
largest_mask = largest_cluster.updateMask(largest_cluster)
m.addLayer(largest_mask, {'min': 0, 'max': 1, 'palette': ['red']}, 'Largest cluster (count == T21_max)')
print('T21_max:', T21_max.getInfo())
print("Min and Max values:", minMax.getInfo())

T21_max: 147
Min and Max values: {'T21_max': 147, 'T21_min': 1}
Min and Max values: {'T21_max': 147, 'T21_min': 1}


In [18]:

# Ensure a geemap Map object exists (create one if needed)
try:
    m  # noqa: F821
except NameError:
    # Use the existing import from earlier cells; do not re-import
    m = geemap.Map(center=[0, 0], zoom=2)

# Polygonize the largest masked cluster and zoom to it
vectors = largest_mask.reduceToVectors(
    reducer=ee.Reducer.countEvery(),
    geometry=geometry,
    scale=1000,
    maxPixels=1e9,
    eightConnected=True,
    geometryType='bb'
)

# Compute area for each polygon and select the largest.
# Specify a non-zero maxError to avoid Geometry.area errors in server-side ops.
def set_area(feat):
    geom = feat.geometry()
    # use a small non-zero maxError (in meters)
    area_m = geom.area(1)
    return feat.set('area', area_m)

vectors = vectors.map(set_area)
largest_feature = ee.Feature(vectors.sort('area', False).first())

# Compute and print the area of the largest_feature
area_m = largest_feature.geometry().area(1)  # area in square meters (server-side)
area_m_val = area_m.getInfo()  # fetch to client
print('Largest feature area (m²):', area_m_val)
print('Largest feature area (km²):', area_m_val / 1e6)

# Add layers and center the map on the largest polygon
m.addLayer(vectors, {'color': 'orange'}, 'Largest cluster polygons')
m.addLayer(ee.FeatureCollection(largest_feature), {'color': 'red'}, 'Largest polygon')
m.centerObject(largest_feature, 10)
m

Largest feature area (m²): 382733153.4267874
Largest feature area (km²): 382.7331534267874


Map(bottom=123257.0, center=[-26.938678339197335, 114.36961163687997], controls=(WidgetControl(options=['posit…

In [19]:
import os
import json
import geojson
from datetime import date

# Fetch the server-side feature to the client as a GeoJSON-like dict
feat_geojson = largest_feature.getInfo()

# Pretty-print GeoJSON to console for copy-paste
geojson_str = json.dumps(feat_geojson, indent=2)
print(geojson_str)

# Ensure output directory exists and write the GeoJSON file using the naming convention
out_dir = "./output"
os.makedirs(out_dir, exist_ok=True)
filename = f"{date.today().isoformat()}_largest_fire_global.geojson"
out_path = os.path.join(out_dir, filename)

with open(out_path, "w") as f:
    f.write(geojson_str)

print(f"\nWritten to: {out_path}")

{
  "type": "Feature",
  "geometry": {
    "geodesic": false,
    "type": "Polygon",
    "coordinates": [
      [
        [
          114.20889998187037,
          -26.992669876477567
        ],
        [
          114.5303232918898,
          -26.992669876477567
        ],
        [
          114.5303232918898,
          -26.88464336276815
        ],
        [
          114.20889998187037,
          -26.88464336276815
        ],
        [
          114.20889998187037,
          -26.992669876477567
        ]
      ]
    ]
  },
  "id": "+31351+13008",
  "properties": {
    "area": 382733153.4267874,
    "count": 147,
    "label": 1
  }
}

Written to: ./output/2025-11-12_largest_fire_global.geojson


## 2. Search and Request Planet Imagery for the Largest Fire

In this section we'll use the AOI derived from the previous section (the server-side feature stored in `largest_feature`) to search Planet's catalog and request high-resolution PlanetScope imagery for detailed inspection.

Planned steps
- Export or convert `largest_feature` to a client-side GeoJSON geometry (or buffer it slightly to create a practical AOI).
- Authenticate to Planet (provide your Planet API key via environment variable or the SDK).
- Build a Planet search using:
    - geometry filter from the AOI
    - date range (e.g., last 7–14 days)
    - cloud-cover filter (e.g., < 20%)
    - item type(s): `PSScene` / `PSScene4Band` (PlanetScope)
- Inspect search results (scene footprint, acquisition date, cloud cover) and pick candidate scenes.
- Place an order using the Planet Orders API (specify item ids, products/bundles, and delivery options).
- Monitor the order and, when ready, visualize delivered assets directly in the notebook (using geemap or by downloading and displaying the images).

Notes and tips
- Keep searches and orders small for quicker turnaround (limit to a few target items).
- Use a BBox with a small buffer around `largest_feature` to ensure full scene coverage.
- Store your Planet API key securely (do not hard-code it in the notebook). Example: export PL_API_KEY in your shell or use a notebook-based prompt to inject it at runtime.
- The next cells will provide concrete code examples for: converting the EE feature to GeoJSON, authenticating to Planet, running a filtered search, and submitting an order.

In [20]:
# You can install packages that aren't currently installed in your Python Notebook using !pip install <package name>
# In this case, we will install the Planet Package:
!pip install rasterio planet folium asyncio requests geemap

In [21]:
#general packages
import os
import json
import glob
import asyncio
import requests
import nest_asyncio
import matplotlib.pyplot as plt
from requests.auth import HTTPBasicAuth
from datetime import datetime, timedelta


#geospatial packages
import rasterio
import geopandas as gpd
from shapely.geometry import shape
from shapely.ops import unary_union
import folium


#planet SDK
from planet import Auth, reporting, Session, OrdersClient, order_request, data_filter

# Google Colab for authentication
# from google.colab import userdata

# We will also create a small helper function to print out JSON with proper indentation.
def indent(data):
    print(json.dumps(data, indent=2))

## Authentication with Environmental Variable

Here, you will paste your API Key when prompted. It will be used to authenticate when ordering data.

Be sure to go to **Edit>Clear all outputs** to clear the console output that results, before sharing this notebook, or uploading it to a public repository, such as GitHub.

Additionally, regularly resetting your API Key on Planet.com can help keep your account and access secure.

You can also authenticate via the CLI using [`auth init`](https://planet-sdk-for-python-v2.readthedocs.io/en/latest/cli/cli-reference/?h=auth#auth:~:text=message%20and%20exit.-,auth,-%C2%B6), this will store your API key as an environment variable.

In [26]:
# if your Planet API Key is not set as an environment variable, you can paste it below
if 'PLANET_API_KEY' in os.environ:
    API_KEY = os.environ['PLANET_API_KEY']
else:
    API_KEY = input("PASTE_API_KEY_HERE AND HIT RETURN:   ")
    os.environ['PLANET_API_KEY'] = API_KEY

client = Auth.from_key(API_KEY)

## Using Google Colab Secret Manager

In [ ]:
from google.colab import userdata
#Store your API key in the Colabs Secret Manager, as PL_API_KEY and enable notebook access to the secret
#Be sure to toggle on Notebook Access in the Secret Manager
# Get the API key from the secret manager
API_KEY = userdata.get('PL_API_KEY')

# Set the API key as an environment variable
os.environ['PL_API_KEY'] = API_KEY

# Create a client for the Planet API
client = Auth.from_key(API_KEY)

## Searching an Area of Interest (AOI)

This code reads a **GeoJSON** file named `"lakelagunita.geojson"`
into a Python object using the `json.loads(f.read())` method, which converts the file's JSON content into a Python dictionary. It then accesses the `'features'` list from this dictionary, selects the first feature (`[0]` indicating the first item in the list), and retrieves the `'geometry'` information associated with that feature. Essentially, it extracts the geometric data (such as coordinates defining points, lines, or polygons) of the first geographic feature described in the GeoJSON file.

The `geom_all` variable stores the geometry of the first feature from the file, demonstrating how to access nested data in JSON format. The `geom_large` dictionary defines a new geometry directly in the code, illustrating how to construct a GeoJSON feature programmatically. You could use either of these methods for providing the **Area of Interest (AOI)**


Let's also read in a GeoJSON geometry into a variable so we can use it during testing. *The geometry can only have one polygon to work with the data API*

In [23]:
with open("./output/2025-11-12_largest_fire_global.geojson") as f:
    geom_all = json.loads(f.read())['geometry']
geom_large = {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
          114.20889998187037,
          -26.992669876477567
        ],
        [
          114.5303232918898,
          -26.992669876477567
        ],
        [
          114.5303232918898,
          -26.88464336276815
        ],
        [
          114.20889998187037,
          -26.88464336276815
        ],
        [
          114.20889998187037,
          -26.992669876477567
        ]
          ]
        ],
        "type": "Polygon"
      }
    }
print(geom_all)

{'geodesic': False, 'type': 'Polygon', 'coordinates': [[[114.20889998187037, -26.992669876477567], [114.5303232918898, -26.992669876477567], [114.5303232918898, -26.88464336276815], [114.20889998187037, -26.88464336276815], [114.20889998187037, -26.992669876477567]]]}


In [ ]:
# Compute bounds from the existing shapely polygon (minx, miny, maxx, maxy)
minx, miny, maxx, maxy = geom_all.bounds
# folium expects [[south, west], [north, east]] i.e. [[min_lat, min_lon], [max_lat, max_lon]]
bounds = [[miny, minx], [maxy, maxx]]

# Center the map on the bbox centroid and create the map using OpenStreetMap basemap
center = [(miny + maxy) / 2, (minx + maxx) / 2]
fm = folium.Map(location=center, zoom_start=10, tiles="OpenStreetMap")

# Add the GeoJSON and layer control, then fit the map to the bbox
folium.GeoJson(geom_all, name="Fire BBox").add_to(fm)
folium.LayerControl().add_to(fm)

# Zoom/fit to the fire bbox extent
fm.fit_bounds(bounds)

fm

# Creating our Filters

This code block sets up filters for querying geospatial data:

- **Item Types**: Defines the type of satellite imagery to search for, in this case, "PSScene".
- **Geometry Filter**: Uses a predefined geometry (`geom_large`) to filter data to only include imagery that intersects with this area.
- **Date Range Filter**: Specifies a date range to filter the imagery, selecting images acquired between December 10, 2022, and September 30, 2023.
- **Combined Filter**: Combines the geometry and date range filters using an "AND" logic, meaning both conditions must be met for an image to be included in the search results.

The cloud cover filter is *commented out*, indicating it's not currently used but can be applied to restrict results to images with less than 80% cloud cover.

The possible filters include `and_filter`, `date_range_filter`, `range_filter` and so on, mirroring the options supported by the Planet API. Additional filters are described [here](https://planet-sdk-for-python-v2.readthedocs.io/en/latest/python/sdk-guide/#filter:~:text=(main())-,Filter,-%C2%B6).

In [24]:
# Define the filters we'll use to find our data

item_types = ["PSScene"]

#Geometry filter
geom_filter = data_filter.geometry_filter(geom_all)

#Date range filter
date_range_filter = data_filter.date_range_filter(
    "acquired", gt = datetime(month=11, day=10, year=2025),
    lt = datetime(month=11, day=11, year=2025))
#Cloud cover filter
#cloud_cover_filter = data_filter.range_filter('clear_percent', gt = 80)

#Combine all of the filters
combined_filter = data_filter.and_filter([geom_filter, date_range_filter])#, cloud_cover_filter])

# Using the SDK

Using the REST API directly requires manually crafting HTTP requests, handling authentication, parsing responses, and managing session states, offering more control and flexibility but requiring more code to handle the interaction.

In contrast, using the SDK (Software Development Kit) abstracts and simplifies the process of interacting with the API by providing pre-built functions and methods, handling low-level details like session management and request retries. It allows for more Pythonic code, with asynchronous capabilities and direct integration into Python applications.

This code performs an asynchronous search request using the Planet SDK, retrieving a list of items that match the specified combined_filter and item_types criteria. It uses an asynchronous session to make the request and asynchronously iterates over the search results up to a limit of 500 items, gathering them into a list called `item_list`. This approach allows for non-blocking network requests, making the code efficient for handling I/O-bound tasks like web requests in a concurrent manner.

If the number of items requested is more than 500, the client will automatically fetch more pages of results in order to get the exact number requested.

Then we can save the output to be visualized as a geojson

In [27]:
async with Session() as sess:
    cl = sess.client('data')
    item_list = [i async for i in cl.search(search_filter=combined_filter, item_types=item_types,limit=500)]

InvalidAPIKey: {"message": "Please provide valid credentials. See https://docs.planet.com/develop/authentication for help.", "errors": []}